In [8]:
import requests
import hashlib
import uuid
import datetime as datetime
import pandas as pd
import hmac
import time
from time import sleep
import sys

In [2]:
client_id = 'YOUR-ID-HERE'
api_key = 'YOUR-API-KEY-HERE'
API_SECRET = b'YOUR-API-SECRET-HERE'

In [4]:
while True:
    try:
        btc_old = requests.get('https://www.bitstamp.net/api/ticker/BTC').json()
        print(btc_old['ask'])
    except excpetion as e:
        print('Error obtaining BTC old data')
        
    sleep(300)
    
    try:
        btc_new = requests.get('https://www.bitstamp.net/api/ticker/BTC').json()
        print(btc_new['ask'])
    except Excpetion as e:
        print('Error obtaining BTC new data')
        
    percent = (((float(btc_new['ask']) - float(btc_old['ask'])) * 100) / float(btc_old['ask']))
    
    if percent < 5:
        print('The trade requirement was not satisfied. The percentage move is at ',percent)
        sleep(5)
        continue
    
    elif percent >= 5:
        try:
            timestamp = str(int(round(time.time() * 1000)))
            nonce = str(uuid.uuid4())
            content_type = 'application/x-www-form-urlencoded'
            payload = {'amount' : 1,
                       'price' : 36000.00,}

            if sys.version_info.major >= 3:
                from urllib.parse import urlencode
            else:
                from urllib import urlencode

            payload_string = urlencode(payload)

            
            message = 'BITSTAMP ' + api_key + \
                'POST' + \
                'www.bitstamp.net' + \
                '/api/v2/buy/market/' + \
                'ETH' + \
                content_type + \
                nonce + \
                timestamp + \
                'v2' + \
                payload_string
            
            message = message.encode('utf-8')
            signature = hmac.new(API_SECRET, msg=message, digestmod=hashlib.sha256).hexdigest()
            headers = {
                'X-Auth': 'BITSTAMP ' + api_key,
                'X-Auth-Signature': signature,
                'X-Auth-Nonce': nonce,
                'X-Auth-Timestamp': timestamp,
                'X-Auth-Version': 'v2',
                'Content-Type': content_type
                }
            
            r = requests.post(
                'https://www.bitstamp.net/api/v2/buy/market/ETH',
                headers=headers,
                data=payload_string
                )
            
            if not r.status_code == 200:
                raise Exception('Status code not 200')

            string_to_sign = (nonce + timestamp + r.headers.get('Content-Type')).encode('utf-8') + r.content
            signature_check = hmac.new(API_SECRET, msg=string_to_sign, digestmod=hashlib.sha256).hexdigest()
            if not r.headers.get('X-Server-Auth-Signature') == signature_check:
                raise Exception('Signatures do not match')

            print(r.content)
        except Exception as e:
                print('Error making order request')
        
        sleep(2)
        order_id = r['content']['id']
        timestamp = str(int(round(time.time() * 1000)))
        new_nonce = str(uuid.uuid4())
        headers = {
                'X-Auth': 'BITSTAMP ' + api_key,
                'X-Auth-Signature': signature,
                'X-Auth-Nonce': new_nonce,
                'X-Auth-Timestamp': timestamp,
                'X-Auth-Version': 'v2',
                'Content-Type': content_type
                }
        try:
            r = requests.post(
                'https://www.bitstamp.net/api/v2/order_status/',
                headers=headers,
                params = {'id': order_id}
                )
        except Excpetion as e:
            print('Error checking for order status')
            
        if r['content']['status'] == 'Open':
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break
        else:
            print('Order was canceled {}'.format(pd.Timestamp.now()))
            break
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-4-80548ff72afc>, line 4)